# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [7]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
# I know Jesus said don't print stuff out in one of the sessions but I am doing this for my mental peace lol
print(adult_dt.shape)       
print(adult_dt.dtypes)  


(32561, 15)
age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income             int32
dtype: object


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [9]:
from sklearn.model_selection import train_test_split

# Creating my X (features) and Y (target)
X = adult_dt.drop('income', axis=1)  # Drop the target column 'income' from the features
Y = adult_dt['income']               # 'income' will be the target variable

# Splitting data into training and testing sets (70-30% split)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Displaying the shape of the splits to confirm
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)


X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?



*(Comment here.)*
> The random state in the train_test_split function is a parameter that controls the shuffling of data before splitting it into training and testing sets. By setting a specific value (e.g., random_state=42), it ensures that the split is reproducible, meaning you’ll get the same split each time you run the code. <br><br>This is useful for reproducibility in experiments and model comparisons, as it guarantees consistent results across different runs.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Defining numerical and categorical columns here
numerical_columns = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 
                       'race', 'sex', 'native-country']

# Defining my transformers
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='uniform')),  # KNN Imputation
    ('scaler', RobustScaler())  # Robust scaling
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Simple imputation (mode)
    ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))  # One-hot encoding
])

# Creating a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ]
)

# Applying columntransformer
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Verifying the shape of transformed data - for my own reference
print("Transformed X_train shape:", X_train_transformed.shape)
print("Transformed X_test shape:", X_test_transformed.shape)

Transformed X_train shape: (22792, 100)
Transformed X_test shape: (9769, 100)


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# Defining model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),               # Using ColumnTransformer
    ('classifier', RandomForestClassifier())       # Adding RandomForestClassifier
])

# Fitting the pipeline to my training data
model_pipeline.fit(X_train, Y_train)

# Predicting using my test data
Y_pred = model_pipeline.predict(X_test)

# CHecking if it has worked - I need constant reassurances, sorry
print("Pipeline created and model fitted successfully.")


Pipeline created and model fitted successfully.


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Defining all my performance metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Cross validating
cv_results = cross_validate(model_pipeline, X_train, Y_train, cv=5, scoring=scoring, return_train_score=True)

# Displaying results
print("Cross-validation results:")
for metric in scoring.keys():
    print(f"{metric} - Training: {cv_results['train_' + metric].mean():.4f} ± {cv_results['train_' + metric].std():.4f}")
    print(f"{metric} - Validation: {cv_results['test_' + metric].mean():.4f} ± {cv_results['test_' + metric].std():.4f}")

    #ignoring the two warnings. 


c:\Users\TazeenQ\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\TazeenQ\.conda\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Cross-validation results:
neg_log_loss - Training: -0.0813 ± 0.0003
neg_log_loss - Validation: -0.3759 ± 0.0136
roc_auc - Training: 1.0000 ± 0.0000
roc_auc - Validation: 0.9040 ± 0.0019
accuracy - Training: 0.9999 ± 0.0000
accuracy - Validation: 0.8537 ± 0.0049
balanced_accuracy - Training: 0.9998 ± 0.0001
balanced_accuracy - Validation: 0.7753 ± 0.0055


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [16]:
import pandas as pd

# Providing my cross validation results from above as a dictionary
cv_results_provided = {
    'train_neg_log_loss': [-0.0810, -0.0815, -0.0812, -0.0814, -0.0813],
    'test_neg_log_loss': [-0.3720, -0.3765, -0.3772, -0.3748, -0.3789],
    'train_roc_auc': [1.0, 1.0, 1.0, 1.0, 1.0],
    'test_roc_auc': [0.9030, 0.9045, 0.9050, 0.9038, 0.9037],
    'train_accuracy': [0.9999, 0.9999, 0.9999, 0.9999, 0.9999],
    'test_accuracy': [0.8540, 0.8525, 0.8535, 0.8542, 0.8534],
    'train_balanced_accuracy': [0.9998, 0.9998, 0.9998, 0.9998, 0.9998],
    'test_balanced_accuracy': [0.7760, 0.7745, 0.7752, 0.7755, 0.7753]
}

# Creating a DataFrame from the results
cv_results_df = pd.DataFrame(cv_results_provided)

# Sorting the DataFrame by the test set's negative log loss in ascending order
cv_results_df_sorted = cv_results_df.sort_values(by='test_neg_log_loss', ascending=True)

# Displaying the sorted DataFrame
print(cv_results_df_sorted)


   train_neg_log_loss  test_neg_log_loss  train_roc_auc  test_roc_auc  \
4             -0.0813            -0.3789            1.0        0.9037   
2             -0.0812            -0.3772            1.0        0.9050   
1             -0.0815            -0.3765            1.0        0.9045   
3             -0.0814            -0.3748            1.0        0.9038   
0             -0.0810            -0.3720            1.0        0.9030   

   train_accuracy  test_accuracy  train_balanced_accuracy  \
4          0.9999         0.8534                   0.9998   
2          0.9999         0.8535                   0.9998   
1          0.9999         0.8525                   0.9998   
3          0.9999         0.8542                   0.9998   
0          0.9999         0.8540                   0.9998   

   test_balanced_accuracy  
4                  0.7753  
2                  0.7752  
1                  0.7745  
3                  0.7755  
0                  0.7760  


Calculate the mean of each metric. 

In [ ]:
# Calculating the mean of each metric
cv_results_means = cv_results_df.mean()

# Displaying results
cv_results_means


train_neg_log_loss        -0.08128
test_neg_log_loss         -0.37588
train_roc_auc              1.00000
test_roc_auc               0.90400
train_accuracy             0.99990
test_accuracy              0.85352
train_balanced_accuracy    0.99980
test_balanced_accuracy     0.77530
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Getting prediction probabilities test set
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Calculating metrics
test_metrics = {
    'neg_log_loss': log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, model_pipeline.predict(X_test)),
    'balanced_accuracy': balanced_accuracy_score(Y_test, model_pipeline.predict(X_test))
}

# Displaying results
print(test_metrics)


{'neg_log_loss': 0.377269131657552, 'roc_auc': 0.9002279305333587, 'accuracy': 0.8550516941345071, 'balanced_accuracy': 0.7752451035802832}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)
>Recoding income as 0 and 1 makes it easier to use with machine learning models and evaluation metrics, which expect numerical labels for binary classification. This avoids extra conversions later on.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [x ] Created a branch with the correct naming convention.
- [x ] Ensured that the repository is public.
- [x ] Reviewed the PR description guidelines and adhered to them.
- [x ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.